In [1]:
import os
from dataclasses import dataclass
from pathlib import Path
from typing import List
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
os.chdir("..")

In [3]:
from dataclasses import dataclass
from pathlib import Path
from typing import List

@dataclass
class EvaluationConfig:
    model_path: Path
    training_data_path: Path
    all_params: dict
    mlflow_uri: str
    image_size: List
    batch_size: int

In [4]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [5]:
class ConfigManager:

    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    def get_evaluation_config(self) -> EvaluationConfig:

        config = self.config.evaluation
        eval_config = EvaluationConfig(
            model_path=config.model_path,
            training_data_path=config.training_data_path,
            all_params=self.params,
            batch_size=self.params.BATCH_SIZE,
            image_size=self.params.IMAGE_SIZE,
            mlflow_uri=os.environ['MLFLOW_TRACKING_URI']
            )
        
        return eval_config

In [9]:
import sys

import tensorflow as tf
import mlflow
import mlflow.keras
from sklearn.metrics import f1_score, recall_score, precision_score
import numpy as np
from urllib.parse import urlparse

from cnnClassifier.exception import CustomException
from cnnClassifier.logger import logging

In [10]:
class Evaluation:

    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.model = self.load_model(self.config.model_path)
        self.valid_generator = None
        self._valid_generator()
        self.scores = self._evaluate_model()

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.image_size[:-1],
            batch_size=self.config.batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data_path,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def _evaluate_model(self):
        loss, accuracy = self.model.evaluate(self.valid_generator)
        y_pred_probs = self.model.predict(self.valid_generator)
        y_pred = np.argmax(y_pred_probs, axis=1)
        y_true = self.valid_generator.classes

        precision = precision_score(y_true, y_pred, average='weighted')
        recall = recall_score(y_true, y_pred, average='weighted')
        f1 = f1_score(y_true, y_pred, average='weighted')

        scores = {
            "loss": loss,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1
        }
        return scores


    def save_score(self):
        save_json(path=Path("scores.json"), data=self.scores)

    def login_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(self.scores)

            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model")
                                            # "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [11]:
try:
    config = ConfigManager()
    evaluation_config = config.get_evaluation_config()
    evaluation = Evaluation(config=evaluation_config)
    evaluation.login_mlflow()
    
except Exception as e:
    error = CustomException(e, sys)
    logging.error(error.error_message)
    raise error

Found 102 images belonging to 2 classes.


2024-07-21 20:48:45.796930: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/7 [===>..........................] - ETA: 1s

2024-07-21 20:48:47.072516: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - 1s 113ms/step


2024/07/21 20:48:48 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
/Users/ngkuissi/miniforge3/envs/chest/lib/python3.8/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [40]:
import keras
print(keras.__version__)
print(tf.__version__)

2.13.1
2.13.0
